In [6]:
from pathlib import Path
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langgraph.graph import MessagesState, StateGraph
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import END
from langgraph.prebuilt import tools_condition
from langchain_core.documents import Document
from langchain.chat_models import init_chat_model
import time
import io
from contextlib import redirect_stdout
from datetime import datetime
from testing.response_eval_tools import set_api_key_from_path, initialize_evaluation_llm, evaluate_model_outputs_from_paths
from pathlib import Path

from qa_pairs.qa_data_loader import load_static_demo_pairs
set_api_key_from_path(Path('./testing/gemini-key'))

################# [ Validate GPU Available ] #################
import torch

assert(torch.cuda.is_available())

##############################################################

chroma_path = "./test_db"
# collection_name = 'test_collection'
collection_name = 'multi_proceedings_test'
embedding_model_name = 'all-MiniLM-L6-v2'


client = chromadb.PersistentClient(path=chroma_path)
embedding_function = SentenceTransformerEmbeddingFunction(model_name=embedding_model_name)


collection = client.get_or_create_collection(name=collection_name, embedding_function=embedding_function)


llm = initialize_evaluation_llm()

# Set up retrieval tool
K = 8  # Default number of documents to retrieve

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    global K
    # Query ChromaDB directly
    results = collection.query(
        query_texts=[query],
        n_results=K,
    )

    # Format results for LangChain compatibility
    retrieved_docs = []
    for i in range(len(results['ids'][0])):
        doc_id = results['ids'][0][i]
        content = results['documents'][0][i]
        metadata = results['metadatas'][0][i] if results['metadatas'][0] else {}


        doc = Document(page_content=content, metadata=metadata)
        retrieved_docs.append(doc)

    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    # print(retrieved_docs,serialized)
    return serialized, retrieved_docs

# Build the graph
graph_builder = StateGraph(MessagesState)

def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# Step 2: Execute the retrieval
tools = ToolNode([retrieve])

# Step 3: Generate a response using the retrieved content
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        """<TONE AND STYLE INSTRUCTIONS>
You are **“GRC Response Assistant.”**
• Voice: clear, concise, helpful, and professional—think a seasoned regulatory analyst explaining something to a busy attorney.
• No corporate jargon, no cheerleading, no emoji.
• Favor short paragraphs and bulleted lists; expand only when the user explicitly asks for more detail.
• Always include pinpoint citations (CPUC docket number, title, page or section) so the answer can be verified instantly.
</TONE AND STYLE INSTRUCTIONS>

<IDENTITY & CORE MISSION>
You are an expert assistant focused **exclusively** on California General Rate Case (GRC) proceedings and related regulatory filings before the CPUC.
Your mission in priority order:
1. **Retrieve** the most relevant records for the user’s query.
2. **Respond** with fact‑based summaries, comparisons, or drafts **grounded 100 % in those records**.
3. **Cite** every factual statement.
4. **Detect inconsistencies** between a user’s new draft filing and past submissions when asked.
5. **Refuse or redirect** any request that falls outside GRC scope (e.g., medical advice, unrelated legal areas).
</IDENTITY & CORE MISSION>

<INPUT CHANNELS & DATA HIERARCHY>
You may draw information from three sources, **in this priority order**:
1. **User‑provided documents** in the current session.
2. **Retrieved context** from the vector database `{docs_content}`.
3. **Your static domain knowledge** (only when the above two sources do not suffice and the fact is uncontroversial).
If adequate information is missing, ask a **specific follow‑up question** rather than speculate.
</INPUT CHANNELS & DATA HIERARCHY>

<RETRIEVAL WORKFLOW FOR EVERY TURN>
1. Read the user’s message.
2. Decide whether additional documents are required.
   • If **yes**, call the `retrieve` tool with a concise search string.
   • If **no**, proceed to form an answer.
3. After retrieval, embed the results (already supplied to you as `{docs_content}`) into your reasoning.
4. Draft the reply following the response formats below.
5. Include full citations **immediately after** each claim (e.g., “(PG&E GRC 2023, Exh. 1, p. 23)”).
6. Return the final answer to the user.  Do **not** expose chain‑of‑thought.
</RETRIEVAL WORKFLOW FOR EVERY TURN>

<RESPONSE TYPES & FORMAT GUIDE>
● **Quick factual Q&A** – short paragraph or numbered list with inline citations.
● **Document comparison / consistency check** – a two‑column table: “Prior Filing” vs “Current Draft,” each row a data point with citations; end with a short summary of discrepancies.
● **Drafting request** (e.g., data request response, testimony snippet):
   1) heading; 2) purpose sentence; 3) body written in CPUC‑compliant style; 4) citation footnote block.
● **High‑level explanation** – brief overview followed by bullet points.
Always end with: “_Let me know if you need deeper detail or additional sources._”
</RESPONSE TYPES & FORMAT GUIDE>

<STRICT SCOPE & REFUSAL POLICY>
Allowed topics: CPUC GRC process, utility revenue requirements, testimony structure, data requests, regulatory citations, comparison across GRC cycles, precedent decisions, compliance timelines.
Disallowed topics: personal medical or financial advice, unrelated legal issues, any non‑CPUC jurisdiction matter, speculative forecasts without documentary support, disallowed content per OpenAI policy.
If the user requests disallowed content, respond with a brief apology and a one‑sentence refusal: “_I’m sorry, but I can’t help with that._”
</STRICT SCOPE & REFUSAL POLICY>

<ACCURACY & HALLUCINATION AVOIDANCE>
• Never fabricate citations, docket numbers, or document titles.
• If a requested fact is missing from the sources, say “_I don’t have that information in the provided documents._” and suggest a follow‑up query or document upload.
• Cross‑check numerical values against at least two retrieved sources when possible.
• For legal interpretations, attribute them: “_According to D.21‑06‑035, the Commission held…_”
</ACCURACY & HALLUCINATION AVOIDANCE>

<META INSTRUCTIONS FOR TOOL USE>
• You may assume the tool `retrieve` returns at most **K=3** items sorted by semantic relevance.
• Do not mention internal tool calls in the user‑facing answer.
• You may ask the user to adjust `K` (e.g., “_Would you like me to broaden the search?_”) if initial context is thin.
</META INSTRUCTIONS FOR TOOL USE>

<FINAL REMINDER>
Stay within the GRC domain, be succinct yet precise, cite everything, and ask for clarification whenever context is insufficient.
"""
        f"Document Context: {docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

# Set up the graph connections
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

# Main loop for interaction
def main():
    show_tool_output = True  # Set to True if you want to see tool outputs
    while True:
        input_message = input()
        if input_message == "exit":
            break

        start_time = time.time()

        for step in graph.stream(
            {"messages": [{"role": "user", "content": input_message}]},
            stream_mode="values",
        ):
            if show_tool_output:
                step["messages"][-1].pretty_print()
            else:
                evnt = step["messages"][-1]
                if (type(evnt) == HumanMessage or type(evnt) == AIMessage) and evnt.content != '':
                    evnt.pretty_print()

        end_time = time.time()

        elapsed_time = end_time - start_time
        print(f"\n========================\n\tLLM call took {elapsed_time:.2f} seconds\n========================\n")


In [7]:
def test_prompts_and_save(prompts):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = Path(
        f"./llm_output/test/llm_responses_with_context_qa_gemini_{timestamp}.txt"
    )

    with output_path.open("w", encoding="utf-8") as f:
        for idx, prompt_text in enumerate(prompts):
            print(f"\n--- Running Prompt {idx+1}/{len(prompts)} ---")
            f.write(f"================= Prompt {idx+1} =================\n")
            f.write(f"PROMPT:\n{prompt_text}\n\n")

            # Initialize conversation state
            state = {"messages": [HumanMessage(content=prompt_text)]}

            # Stream and log each step's output, capturing pretty_print to file
            for step in graph.stream(
                state,
                stream_mode="values",
            ):
                last_msg = step["messages"][-1]

                # Capture pretty_print output to buffer
                buffer = io.StringIO()
                with redirect_stdout(buffer):
                    last_msg.pretty_print()
                formatted = buffer.getvalue()

                # Write the formatted message to the file
                f.write(formatted)

            f.write("\n")
            print(f"Prompt {idx+1} complete.")

    print(f"\nAll responses and contexts saved to: {output_path.resolve()}")


In [8]:
test_list = ["What ratemaking mechanism did the CPUC authorize for PG&E's energization projects?"]

In [9]:
test_prompts_and_save(test_list)


--- Running Prompt 1/1 ---
Prompt 1 complete.

All responses and contexts saved to: C:\Users\Elijah\PycharmProjects\GRCResponder_tools\llm_output\test\llm_responses_with_context_qa_gemini_20250429_183206.txt


# Q&A Testing

In [2]:
load_static_demo_pairs(Path("./qa_pairs/A2106021/qa_data"))

Saved 22 Q&A pairs to legal_qa_dataset.json

Loaded 22 Q&A pairs successfully!

Sample Q&A pair:


[{'id': 1,
  'question': 'What is Senate Bill 410 and what is its primary purpose regarding energization projects?',
  'answer': 'Senate Bill 410, the "Powering Up Californians Act," authorizes electrical utilities to recover costs for energization projects that exceed authorized revenue requirements. Its primary purpose is to reduce greenhouse gas emissions by enabling substantial upgrades to the electrical distribution system, allowing new customers to be promptly connected to the grid, existing customers to have their service level upgraded, and proactive planning for increased distribution capacity to accommodate future load.'},
 {'id': 2,
  'question': 'What does Pub. Util. Code Section 937(b) require regarding ratemaking mechanisms for energization projects?',
  'answer': 'Section 937(b) requires the Commission to: 1) authorize tracking of costs for energization projects placed in service after January 1, 2024 that exceed authorized revenue requirements; 2) establish an up-front 

In [19]:
test_list = [
    # General Rate Case Scope & Background
    "What was the total revenue requirement PG&E requested in its 2023 General Rate Case application, and over what period was this increase proposed?",
    "How did PG&E’s 2023 GRC proposal affect the average monthly bill for a typical residential customer?",
    "What major categories of investment did PG&E seek funding for in the 2023 GRC (e.g., wildfire risk mitigation, infrastructure upgrades, gas operations)?",

    # Regulatory Decisions & Implementation
    "What did the CPUC’s final decision in A.21-06-021 authorize for PG&E’s operational and infrastructure revenue requirement for 2023-2026?",
    "What are the next steps PG&E must take following the CPUC’s final decision in its 2023 GRC, and what is the timeline for implementing new rates?",
    "How does the GT&S CARD proceeding relate to the GRC, and when will its cost allocation changes be reflected in customer rates?",

    # Technical/Financial Details
    "What are the updated marginal costs proposed by PG&E for its RTP Pilots in the 2023 GRC Phase II, and how do they compare to previous values?",
    "Describe the rationale for PG&E’s approach to updating marginal costs for RTP Pilots in its 2023 GRC Phase II filing.",

    # Procedural & Compliance Prompts
    "What documents and data must PG&E submit to the CPUC as part of the GRC process, and what are the compliance requirements for these filings?",
    "What is the purpose of the Tier 1 Advice Letter PG&E must file after the GRC final decision, and what information must it contain?",
    "How does the consolidation of gas transmission and storage (GT&S) into the GRC impact the scope and process of the proceeding?",

    # Edge Cases & Error Handling
    "If the CARD settlement decision is delayed, how does that affect the implementation of new rates for PG&E customers?",
    "What happens if PG&E’s Advice Letter does not reflect the updated throughput forecast from the CARD settlement?",

    # User Experience & Contextual Understanding
    "Summarize the key changes for PG&E customers as a result of the 2023 GRC decision.",
    "Explain the difference between Phase I and Phase II of PG&E’s 2023 GRC and their respective focuses."
]


In [13]:
test_list = [ "What was the total revenue requirement PG&E requested in its 2023 General Rate Case application, and what was the increase over 2022?",

"In what year did the Commission combine gas transmission and storage revenue requirements with the General Rate Case, and why was this change made?",

"How does PG&E propose to file updates related to the Gas Storage Balancing Account after the GRC decision?",

"Explain the relationship between the GRC Phase 1 and GT&S CARD proceedings, and what happens after decisions in both proceedings.",

"What are the key differences between how gas transmission and storage costs were handled before and after the 2023 GRC?",

"Based on the available context, what evidence is there that the CPUC considered climate resilience in the 2023 GRC proceedings?",

"What concerns did TURN express about PG&E's 2023 GRC request, and how did they characterize the magnitude of the request?",

"What is the typical timeline for implementing new rates after both the GRC and GT&S CARD decisions are issued?",

"How does PG&E propose to handle over-collections or under-collections in its balancing accounts, and what alternatives did other parties suggest?" ]

In [10]:
test_list = [
    "Based on the 2023 Safety Performance Metrics Report submitted by PG&E to the California Public Utilities Commission on April 1, 2024, summarize the purpose of this report, what safety metrics PG&E is required to track, how these metrics are used to improve safety performance, and any notable safety trends or improvements mentioned in the introduction. Please include information about how these metrics relate to executive compensation and risk-informed decision making."
]

In [14]:
test_prompts_and_save(test_list)


--- Running Prompt 1/9 ---
Prompt 1 complete.

--- Running Prompt 2/9 ---
Prompt 2 complete.

--- Running Prompt 3/9 ---
Prompt 3 complete.

--- Running Prompt 4/9 ---
Prompt 4 complete.

--- Running Prompt 5/9 ---
Prompt 5 complete.

--- Running Prompt 6/9 ---
Prompt 6 complete.

--- Running Prompt 7/9 ---
Prompt 7 complete.

--- Running Prompt 8/9 ---
Prompt 8 complete.

--- Running Prompt 9/9 ---
Prompt 9 complete.

All responses and contexts saved to: C:\Users\Elijah\PycharmProjects\GRCResponder_tools\llm_output\test\llm_responses_with_context_20250419_230346.txt


In [2]:
from testing.response_eval_tools import set_api_key_from_path, initialize_evaluation_llm, evaluate_model_outputs_from_paths
from pathlib import Path
set_api_key_from_path(Path('./testing/gemini-key'))
llm = initialize_evaluation_llm()

---

In [7]:
llm.invoke("""The following text is llm generated. Evaluate how well it utilizes the provided context:

================= Prompt 1 =================
PROMPT:
Based on the 2023 Safety Performance Metrics Report submitted by PG&E to the California Public Utilities Commission on April 1, 2024, summarize the purpose of this report, what safety metrics PG&E is required to track, how these metrics are used to improve safety performance, and any notable safety trends or improvements mentioned in the introduction. Please include information about how these metrics relate to executive compensation and risk-informed decision making.

================================[1m Human Message [0m=================================

Based on the 2023 Safety Performance Metrics Report submitted by PG&E to the California Public Utilities Commission on April 1, 2024, summarize the purpose of this report, what safety metrics PG&E is required to track, how these metrics are used to improve safety performance, and any notable safety trends or improvements mentioned in the introduction. Please include information about how these metrics relate to executive compensation and risk-informed decision making.
==================================[1m Ai Message [0m==================================
Tool Calls:
  retrieve (61bb264c-fb94-466f-beae-8a29dbba1dfd)
 Call ID: 61bb264c-fb94-466f-beae-8a29dbba1dfd
  Args:
    query: The 2023 Safety Performance Metrics Report submitted by PG \&E to the California Public Utilities Commission on April 1, 2024
=================================[1m Tool Message [0m=================================
Name: retrieve

Source: {'chunk_index': 2, 'document_id': '562084597', 'proceeding_id': 'A2006012', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M562/K084/562084597.PDF', 'text': 'DECISIONS 19-04-020 AND 21-11-009\nPacific Gas and Electric Company (PG&E) submits its 2024 Safety Performance Metrics\nReport in compliance with Decisions (D.) 19-04-020 and 21-11-009.\nIn 19-04-020, the Phase Two Decision Adopting Risk Spending Accountability Report\nRequirements and Safety Performance Metrics For Investor-Owned Utilities And Adopting A\nSafety Model Approach For Small And Multi-Jurisdictional Utilities, the California Public\nUtilities Commission (Commission) directed the large investor owned utilities to annually file a\nSafety Performance Metrics Report on March 31.1 The Safety Performance Metrics Report must\ninclude:\n• The last ten years for all Safety Performance Metrics for which such data exists;\n• A narrative context about the value of the safety metrics;\n1 D.19-04-020, p. 26.\n- 1 -\n• Identification of the metrics linked to or used for purposes of determining\nexecutive compensation levels for positions director-level and above;'}
Content: DECISIONS 19-04-020 AND 21-11-009
Pacific Gas and Electric Company (PG&E) submits its 2024 Safety Performance Metrics
Report in compliance with Decisions (D.) 19-04-020 and 21-11-009.
In 19-04-020, the Phase Two Decision Adopting Risk Spending Accountability Report
Requirements and Safety Performance Metrics For Investor-Owned Utilities And Adopting A
Safety Model Approach For Small And Multi-Jurisdictional Utilities, the California Public
Utilities Commission (Commission) directed the large investor owned utilities to annually file a
Safety Performance Metrics Report on March 31.1 The Safety Performance Metrics Report must
include:
• The last ten years for all Safety Performance Metrics for which such data exists;
• A narrative context about the value of the safety metrics;
1 D.19-04-020, p. 26.
- 1 -
• Identification of the metrics linked to or used for purposes of determining
executive compensation levels for positions director-level and above;

Source: {'chunk_index': 2, 'document_id': '528888453', 'proceeding_id': 'A2006012', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M528/K888/528888453.PDF', 'text': 'DECISIONS 19-04-020 AND 21-11-009\nPacific Gas and Electric Company (PG&E) submits its 2023 Safety Performance Metrics\nReport in compliance with Decisions (D.) 19-04-020 and 21-11-009.\nIn 19-04-020, the Phase Two Decision Adopting Risk Spending Accountability Report\nRequirements and Safety Performance Metrics For Investor-Owned Utilities And Adopting A\nSafety Model Approach For Small And Multi-Jurisdictional Utilities, the California Public\nUtilities Commission (Commission) directed the large investor owned utilities to annually file a\nSafety Performance Metrics Report on March 31.1 The Safety Performance Metrics Report must\ninclude:\n\uf0b7 The last ten years for all Safety Performance Metrics for which such data exists;\n\uf0b7 A narrative context about the value of the safety metrics;\n1 D.19-04-020, p. 26.\n- 1 -\n\uf0b7 Identification of the metrics linked to or used for purposes of determining\nexecutive compensation levels for positions director-level and above;'}
Content: DECISIONS 19-04-020 AND 21-11-009
Pacific Gas and Electric Company (PG&E) submits its 2023 Safety Performance Metrics
Report in compliance with Decisions (D.) 19-04-020 and 21-11-009.
In 19-04-020, the Phase Two Decision Adopting Risk Spending Accountability Report
Requirements and Safety Performance Metrics For Investor-Owned Utilities And Adopting A
Safety Model Approach For Small And Multi-Jurisdictional Utilities, the California Public
Utilities Commission (Commission) directed the large investor owned utilities to annually file a
Safety Performance Metrics Report on March 31.1 The Safety Performance Metrics Report must
include:
 The last ten years for all Safety Performance Metrics for which such data exists;
 A narrative context about the value of the safety metrics;
1 D.19-04-020, p. 26.
- 1 -
 Identification of the metrics linked to or used for purposes of determining
executive compensation levels for positions director-level and above;

Source: {'chunk_index': 2, 'document_id': '506169959', 'proceeding_id': 'A2006012', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M506/K169/506169959.PDF', 'text': 'DECISIONS 19-04-020 AND 21-11-009\nPacific Gas and Electric Company (PG&E) submits its 2022 Safety Performance Metrics\nReport in compliance with Decisions (D.) 19-04-020 and 21-11-009.\nIn 19-04-020, the Phase Two Decision Adopting Risk Spending Accountability Report\nRequirements and Safety Performance Metrics For Investor-Owned Utilities And Adopting A\nSafety Model Approach For Small And Multi-Jurisdictional Utilities, the California Public\nUtilities Commission (Commission) directed the large investor owned utilities to annually file a\nSafety Performance Metrics Report on March 31.1 The Safety Performance Metrics Report must\ninclude:\n\uf0b7 The last ten years for all Safety Performance Metrics for which such data exists;\n\uf0b7 A narrative context about the value of the safety metrics;\n1 D.19-04-020, p. 26.\n- 1 -\n\uf0b7 Identification of the metrics linked to or used for purposes of determining\nexecutive compensation levels for positions director-level and above;'}
Content: DECISIONS 19-04-020 AND 21-11-009
Pacific Gas and Electric Company (PG&E) submits its 2022 Safety Performance Metrics
Report in compliance with Decisions (D.) 19-04-020 and 21-11-009.
In 19-04-020, the Phase Two Decision Adopting Risk Spending Accountability Report
Requirements and Safety Performance Metrics For Investor-Owned Utilities And Adopting A
Safety Model Approach For Small And Multi-Jurisdictional Utilities, the California Public
Utilities Commission (Commission) directed the large investor owned utilities to annually file a
Safety Performance Metrics Report on March 31.1 The Safety Performance Metrics Report must
include:
 The last ten years for all Safety Performance Metrics for which such data exists;
 A narrative context about the value of the safety metrics;
1 D.19-04-020, p. 26.
- 1 -
 Identification of the metrics linked to or used for purposes of determining
executive compensation levels for positions director-level and above;

Source: {'chunk_index': 3, 'document_id': '506169959', 'proceeding_id': 'A2006012', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M506/K169/506169959.PDF', 'text': '\uf0b7 Descriptions of bias controls that the utility has in place for reporting of the\nmetrics;\n\uf0b7 Examples of how the metrics have informed training and supported risk-informed\ndecision-making;\n\uf0b7 Explanations of how the metrics reflect progress against safety goals included in\nthe utility’s General Rate Case; and\n\uf0b7 A high-level summary of the total estimated and recorded risk-related spend.2\nIn the Order Instituting Rulemaking to Further Develop a Risk-Based Decision-Making\nFramework for Electric and Gas Utilities, the Commission reassessed the Safety Performance\nMetrics adopted in D.19-04-020.3 At the conclusion of Phase I of that proceeding, the\nCommission adopted 32 Safety Performance Metrics in D.21-11-009. The report attached hereto\ncovers the revised set of Safety Performance Metrics.\nPG&E’s 2022 Safety Performance Metrics Report is provided as the Attachment.\nRespectfully Submitted,\nBy: /s/ Steven Frank\nSTEVEN FRANK\nPacific Gas and Electric Company\nLaw Department\n300 Lakeside Drive'}
Content:  Descriptions of bias controls that the utility has in place for reporting of the
metrics;
 Examples of how the metrics have informed training and supported risk-informed
decision-making;
 Explanations of how the metrics reflect progress against safety goals included in
the utility’s General Rate Case; and
 A high-level summary of the total estimated and recorded risk-related spend.2
In the Order Instituting Rulemaking to Further Develop a Risk-Based Decision-Making
Framework for Electric and Gas Utilities, the Commission reassessed the Safety Performance
Metrics adopted in D.19-04-020.3 At the conclusion of Phase I of that proceeding, the
Commission adopted 32 Safety Performance Metrics in D.21-11-009. The report attached hereto
covers the revised set of Safety Performance Metrics.
PG&E’s 2022 Safety Performance Metrics Report is provided as the Attachment.
Respectfully Submitted,
By: /s/ Steven Frank
STEVEN FRANK
Pacific Gas and Electric Company
Law Department
300 Lakeside Drive

Source: {'chunk_index': 12, 'document_id': '501533409', 'proceeding_id': 'A2106021', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M501/K533/501533409.PDF', 'text': 'V. CONCLUSION\nIt bears repeating that Commissioner John Reynolds stated in his opening\nremarks, “…Californian’s budgets are not unlimited. We need utilities to prioritize safety\nprojects to maximize the effective use of ratepayer dollars to achieve risk reduction. As\nwe consider this challenge, we must be attentive to false precision where there is\nuncertainty surrounding the key measures of risk, cost, and project timing. Californians\ndeserve operational excellence from their utilities, and operational excellence requires\n18\nPG&E Reply Brief, p. 371.\n19\nPG&E-14, pp. 1-3, lines 13 to 16; Supra fn. 17.\n6\n20\nexcellence in planning, including accounting for uncertainty.” PG&E’s failure to meet its\n2022 undergrounding goals and its recognition of the unreasonableness of its February\n2022 undergrounding proposal provides the Commission all the evidence necessary that\nthe program is not ready for prime time.\nThe cost to ratepayers and the current affordability crisis which ratepayers are'}
Content: V. CONCLUSION
It bears repeating that Commissioner John Reynolds stated in his opening
remarks, “…Californian’s budgets are not unlimited. We need utilities to prioritize safety
projects to maximize the effective use of ratepayer dollars to achieve risk reduction. As
we consider this challenge, we must be attentive to false precision where there is
uncertainty surrounding the key measures of risk, cost, and project timing. Californians
deserve operational excellence from their utilities, and operational excellence requires
18
PG&E Reply Brief, p. 371.
19
PG&E-14, pp. 1-3, lines 13 to 16; Supra fn. 17.
6
20
excellence in planning, including accounting for uncertainty.” PG&E’s failure to meet its
2022 undergrounding goals and its recognition of the unreasonableness of its February
2022 undergrounding proposal provides the Commission all the evidence necessary that
the program is not ready for prime time.
The cost to ratepayers and the current affordability crisis which ratepayers are

Source: {'chunk_index': 3, 'document_id': '528888453', 'proceeding_id': 'A2006012', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M528/K888/528888453.PDF', 'text': '\uf0b7 Descriptions of bias controls that the utility has in place for reporting of the\nmetrics;\n\uf0b7 Examples of how the metrics have informed training and supported risk-informed\ndecision-making;\n\uf0b7 Explanations of how the metrics reflect progress against safety goals included in\nthe utility’s General Rate Case; and\n\uf0b7 A high-level summary of the total estimated and recorded risk-related spend.2\nIn the Order Instituting Rulemaking to Further Develop a Risk-Based Decision-Making\nFramework for Electric and Gas Utilities, the Commission reassessed the Safety Performance\nMetrics adopted in D.19-04-020.3 At the conclusion of Phase I of that proceeding, the\nCommission adopted 32 Safety Performance Metrics in D.21-11-009. The report attached hereto\ncovers the revised set of Safety Performance Metrics.\nPG&E’s 2023 Safety Performance Metrics Report is provided as the Attachment.\nRespectfully Submitted,\nBy: /s/ Peter Ouborg\nPETER OURBORG\nPACIFIC GAS AND ELECTRIC COMPANY\nLaw Department'}
Content:  Descriptions of bias controls that the utility has in place for reporting of the
metrics;
 Examples of how the metrics have informed training and supported risk-informed
decision-making;
 Explanations of how the metrics reflect progress against safety goals included in
the utility’s General Rate Case; and
 A high-level summary of the total estimated and recorded risk-related spend.2
In the Order Instituting Rulemaking to Further Develop a Risk-Based Decision-Making
Framework for Electric and Gas Utilities, the Commission reassessed the Safety Performance
Metrics adopted in D.19-04-020.3 At the conclusion of Phase I of that proceeding, the
Commission adopted 32 Safety Performance Metrics in D.21-11-009. The report attached hereto
covers the revised set of Safety Performance Metrics.
PG&E’s 2023 Safety Performance Metrics Report is provided as the Attachment.
Respectfully Submitted,
By: /s/ Peter Ouborg
PETER OURBORG
PACIFIC GAS AND ELECTRIC COMPANY
Law Department

Source: {'chunk_index': 3, 'document_id': '562084597', 'proceeding_id': 'A2006012', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M562/K084/562084597.PDF', 'text': '• Descriptions of bias controls that the utility has in place for reporting of the\nmetrics;\n• Examples of how the metrics have informed training and supported risk-informed\ndecision-making;\n• Explanations of how the metrics reflect progress against safety goals included in\nthe utility’s General Rate Case; and\n• A high-level summary of the total estimated and recorded risk-related spend.2\nIn the Order Instituting Rulemaking to Further Develop a Risk-Based Decision-Making\nFramework for Electric and Gas Utilities, the Commission reassessed the Safety Performance\nMetrics adopted in D.19-04-020.3 At the conclusion of Phase I of that proceeding, the\nCommission adopted 32 Safety Performance Metrics in D.21-11-009. The report attached hereto\ncovers the revised set of Safety Performance Metrics.\nPG&E’s 2024 Safety Performance Metrics Report is provided as the Attachment.\nRespectfully Submitted,\nBy: /s/ Peter Ouborg\nPETER OURBORG\nPacific Gas and Electric Company\nLaw Department'}
Content: • Descriptions of bias controls that the utility has in place for reporting of the
metrics;
• Examples of how the metrics have informed training and supported risk-informed
decision-making;
• Explanations of how the metrics reflect progress against safety goals included in
the utility’s General Rate Case; and
• A high-level summary of the total estimated and recorded risk-related spend.2
In the Order Instituting Rulemaking to Further Develop a Risk-Based Decision-Making
Framework for Electric and Gas Utilities, the Commission reassessed the Safety Performance
Metrics adopted in D.19-04-020.3 At the conclusion of Phase I of that proceeding, the
Commission adopted 32 Safety Performance Metrics in D.21-11-009. The report attached hereto
covers the revised set of Safety Performance Metrics.
PG&E’s 2024 Safety Performance Metrics Report is provided as the Attachment.
Respectfully Submitted,
By: /s/ Peter Ouborg
PETER OURBORG
Pacific Gas and Electric Company
Law Department

Source: {'chunk_index': 120, 'document_id': '498802272', 'proceeding_id': 'A2106021', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M498/K802/498802272.PDF', 'text': 'costs are just and reasonable.\n1.4.4 Utility Rate Components\nThe Commission must review all components of PG&E’s requests to ensure they are\ngrounded in evidence, achievable, and necessary. Cal Advocates recommendations in its\nExhibits for this TY 2023 GRC provide the Commission with the relevant evidence, expert\nassessment and documented history to make this decision.\n75\nElectrical System Safety: California’s Oversight of the Efforts by Investor Owned Utilities to\nMitigate the Risk of Wildfires Needs Improvement, Cal. State. Auditor (March 2022), Figure 1 at 6.\n(See Cal Advocates Motion for Official Notice)\n76\nElectrical System Safety: California’s Oversight of the Efforts by Investor Owned Utilities to\nMitigate the Risk of Wildfires Needs Improvement, Cal. State. Auditor (March 2022), Figure 1 at 3.\n(See Cal Advocates Motion for Official Notice).\n77\nEx. CalPA-17: Public Advocates Office Report on the Results of Operations for Pacific Gas and'}
Content: costs are just and reasonable.
1.4.4 Utility Rate Components
The Commission must review all components of PG&E’s requests to ensure they are
grounded in evidence, achievable, and necessary. Cal Advocates recommendations in its
Exhibits for this TY 2023 GRC provide the Commission with the relevant evidence, expert
assessment and documented history to make this decision.
75
Electrical System Safety: California’s Oversight of the Efforts by Investor Owned Utilities to
Mitigate the Risk of Wildfires Needs Improvement, Cal. State. Auditor (March 2022), Figure 1 at 6.
(See Cal Advocates Motion for Official Notice)
76
Electrical System Safety: California’s Oversight of the Efforts by Investor Owned Utilities to
Mitigate the Risk of Wildfires Needs Improvement, Cal. State. Auditor (March 2022), Figure 1 at 3.
(See Cal Advocates Motion for Official Notice).
77
Ex. CalPA-17: Public Advocates Office Report on the Results of Operations for Pacific Gas and
================================= Ai Message ==================================

Based on the 2023 Safety Performance Metrics Report submitted by PG&E to the California Public Utilities Commission on April 1, 2024, here is a summary of the report's content:

**Purpose of the Report:**
The purpose of this report is to provide a comprehensive overview of PG&E's safety performance in 2023, as required by California law (Public Utility Safety Commission Rule 14, CCR § 1600 et seq.). The report aims to inform the public about PG&E's commitment to safety and its progress in achieving its safety goals.

**Required Safety Metrics:**
PG&E is required to track a set of safety metrics, including:

1. Work-related injuries (WRIs) and illnesses
2. Fatalities
3. Near misses (non-fatal injuries or illnesses that could have resulted in a WRi)
4. Environmental releases
5. Electrical fatalities and injuries

These metrics are used to assess PG&E's overall safety performance and identify areas for improvement.

**Improving Safety Performance:**
The report highlights several initiatives aimed at improving safety performance, including:

1. Enhanced training programs for employees
2. Improved fall protection systems
3. Increased use of personal protective equipment (PPE)
4. Enhanced incident investigation processes

These efforts aim to reduce the number of WRIs, fatalities, and near misses, while also minimizing environmental releases.

**Notable Safety Trends or Improvements:**
The report mentions several notable safety trends or improvements, including:

1. A 15% decrease in Work-related injuries (WRIs) compared to the previous year
2. A 25% decrease in electrical fatalities compared to the previous year
3. Improved incident investigation processes, resulting in more effective root cause analysis and corrective actions

**Relation to Executive Compensation:**
The report notes that PG&E's safety performance metrics are included in the company's executive compensation plan as a key performance indicator (KPI). This means that executives' compensation is tied to their ability to improve safety performance, which incentivizes them to focus on reducing risks and improving safety outcomes.

**Relation to Risk-Informed Decision Making:**
The report highlights how PG&E uses its safety metrics to inform risk-informed decision making. By analyzing these metrics, the company can identify areas where safety risks are highest and prioritize resources accordingly. This approach enables PG&E to make more informed decisions about investments, operations, and regulatory submissions.

Overall, the 2023 Safety Performance Metrics Report provides a comprehensive overview of PG&E's safety performance in 2023 and highlights several initiatives aimed at improving safety outcomes. The report also underscores the importance of tracking safety metrics as a means of incentivizing executive compensation and informing risk-informed decision making.


""")

AIMessage(content="The response does not accurately utilize the provided context.\n\nStrengths:\n*   The response correctly identifies the purpose of the report.\n*   It mentions that the report includes metrics linked to executive compensation.\n*   It correctly states the report is submitted to the California Public Utilities Commission (CPUC).\n\nWeaknesses:\n*   **Incorrect Date:** The response states the report was submitted on April 1, 2024, which is correct according to the prompt, however the model hallucinates that this date is a requirement of California law.\n*   **Hallucinated Content:** The response hallucinates safety metrics and improvements. The provided context only contains information about the *requirements* of the report, not the actual data within the report. The response fabricates specific metrics and improvements that are nowhere to be found in the context.\n*   **Misinterpretation of Context:** The response mentions that PG&E's safety performance metrics are i